Import **random** library for generating random variables.

Import **math** library for floor operation.

Import **csv** library for writing the outcomes to .csv files

In [ ]:
import random
import math
import csv

**Generate function for random variables is defined.**

In [ ]:
def Generate_Interarrival():
    return random.expovariate(1.0) # 1 patient/hour is given in the description


def Generate_Nurse_Service_Time():
    return random.expovariate(TRIAGE_NURSE_RATE)


def Generate_Hospital_Healing_Time():
    return random.expovariate(HEALING_IN_HOSPITAL_RATE)


def Generate_Home_Healing_Time(condition):
    if condition == "S":
        return random.expovariate(HEALING_STABLE_IN_HOME_RATE)
    else:
        return random.expovariate(HEALING_IN_HOSPITAL_RATE / random.uniform(1.25, 1.75))  # Should we add +1?

Parameters that come from **moodle** for **Group 3**

In [ ]:
# Group 3
random.seed(2018400372 + 2019400288)
S = 3  # number of nurses
TRIAGE_NURSE_RATE = 0.476190476  # The evaluation time of an arriving patient by a triage nurse
P_1 = 0.2  # probability of being in stable condition.
P_2 = 1.0 - P_1  # probability of being in critical condition.
K = 9 # number of beds
HEALING_IN_HOSPITAL_RATE = 0.118518519

HEALING_STABLE_IN_HOME_RATE = 0.16  # in hour

Parameters that are related to calculating **the event and model response**

In [ ]:
current_time = 0.0

# Arrival()
total_number_of_arrival = 0  # Arrival to triage #total number of sick people
total_number_of_people_in_queue = 0

# Departure_Triage()
total_number_of_departure_from_triage = 0  # Departure from triage to home or bed.
total_number_of_rejected_due_to_bed = 0  # for average number of people that are rejected due to bed unavailability
total_time_spent_in_queue = 0.0
total_number_of_critical_patients = 0
total_number_of_stable_patients = 0

# Treated_at_Hospital()
total_time_healing_sick = 0.0  # for average time a sick person gets better.
total_number_of_people_treated_in_hospital = 0

# Treated_at_Home()
total_number_of_people_treated_in_home = 0  # for average number of patients that are treated at home

# Execute_Event()
total_number_of_executed_events = 0

# Nurse
queue_for_triage_nurse = []
is_nurse_available = [True] * S
nurse_service_begin_time = [0.0] * S
total_busy_time_for_nurse = [0.0] * S  # for average utilization of each triage nurse

# Bed
is_bed_available = [True] * K
bed_service_begin_time = [0.0] * K
total_busy_time_for_bed = [0.0] * K

# Probability variables
total_time_there_is_atleast_one_empty_nurse = 0.0
total_time_there_is_atleast_one_empty_bed = 0.0
total_time_there_is_atleast_one_empty_nurse_and_bed = 0.0

# Future event List
future_event_list = []

**Headers of the columns** for the **event.csv** file is calculated here

In [ ]:
csv_fields=["Event ID","Clock","Future Event List","People In Queue","Nurse Availabilities","Bed Availabilities"]
csv_fields.append("Total Number Of Patients") #Arrival
csv_fields.append("Total Number Of People In Queue")
csv_fields.append("Total Number Of Departure From Triage")
csv_fields.append("Total Number Of Rejected People Due To Bed")
csv_fields.append("Total Time Healing Sick")
csv_fields.append("Total Time Spent In Queue")
csv_fields.append("Total Number Of Critical Patients")
csv_fields.append("Total Number Of Stable Patients")
csv_fields.append("Total Number Of People Treated In Hospital")
csv_fields.append("Total Number Of People Treated In Home")

**Future Event List** will have the following **Events**
- Arrival
- Departure from Nurse
- Departure from Bed
- Departure from Home
1. The type field can be A,DH,DNX,DBX where X can be 0,1,2 ... depending on the number of nurses and beds
2. The time field indicates the occurence time of the event.
3. The ID field indicates the event's id.

In [ ]:
class Event:
    def __init__(self, type, id, time):
        self.type = type  # A=Arrival to triage, DNX=Departure from Nurse X, DBX=Departure from Bed X, DH=Departure from Home
        self.id = id
        self.time = time

    def __repr__(self):
        return f"Type:{self.type} ID:{self.id} Time:{self.time}"

The following function will be use for **event.csv** file to **print current state of the system at any time**.

In [ ]:
def printCurrentEvent(csv_file_name):
    global current_time
    global total_number_of_executed_events
    global future_event_list
    global is_nurse_available
    global is_bed_available
    global total_number_of_arrival  # Arrival to triage
    global total_number_of_people_in_queue
    global total_number_of_departure_from_triage
    global total_number_of_rejected_due_to_bed  # for average number of people that are rejected due to bed unavailability
    global total_number_of_people_treated_in_home  # for average number of patients that are treated at home
    global total_time_healing_sick  # for average time a sick person gets better.
    global total_time_spent_in_queue
    global total_number_of_critical_patients
    global total_number_of_stable_patients
    global total_number_of_people_treated_in_hospital
    csv_row=[total_number_of_executed_events,current_time,future_event_list,queue_for_triage_nurse,is_nurse_available,is_bed_available]
    csv_row.append(total_number_of_arrival)
    csv_row.append(total_number_of_people_in_queue)
    csv_row.append(total_number_of_departure_from_triage)
    csv_row.append(total_number_of_rejected_due_to_bed)
    csv_row.append(total_time_healing_sick)
    csv_row.append(total_time_spent_in_queue)
    csv_row.append(total_number_of_critical_patients)
    csv_row.append(total_number_of_stable_patients)
    csv_row.append(total_number_of_people_treated_in_hospital)
    csv_row.append(total_number_of_people_treated_in_home)
    with open(csv_file_name,"a") as csv_file:
        csv.writer(csv_file).writerow(csv_row)

The following functions is used to simulate the system and related to **Part 1.2** in the description. Only Treated_at_Home() is not mentioned in the description. It basically does the event for critical or stable patient to be healed in the home.
- Arrival()
- Departure Triage()
- Treated_at_Hospital()
- Treated_at_Home()
- Advance_Time()
- Execute_Event()

In [ ]:

# Arrival-to-Triage
def Arrival():
    global future_event_list
    global current_time
    global total_number_of_arrival
    global nurse_service_begin_time
    global total_number_of_people_in_queue

    # for scheduling next departure from triage nurse
    Found = False
    for i in range(S):
        if is_nurse_available[i]:
            is_nurse_available[i] = False
            nurse_service_begin_time[i] = current_time
            next_departure_time = current_time + Generate_Nurse_Service_Time()
            future_event_list.append(Event("DN" + str(i), future_event_list[0].id, next_departure_time))
            Found = True
            break
    if not Found:
        total_number_of_people_in_queue += 1
        queue_for_triage_nurse.append(future_event_list[0])
    total_number_of_arrival += 1

    # for scheduling next arrival
    next_arrival_time = Generate_Interarrival() + future_event_list[0].time
    next_arrival = Event("A", total_number_of_arrival + 1, next_arrival_time)
    future_event_list.append(next_arrival)

    future_event_list.pop(0)


# Triage-to-Bed
def Departure_Triage():
    global future_event_list
    global queue_for_triage_nurse
    global is_nurse_available
    global total_busy_time_for_nurse
    global nurse_service_begin_time
    global current_time
    global is_bed_available
    global total_number_of_rejected_due_to_bed
    global total_number_of_departure_from_triage
    global total_number_of_people_treated_in_home
    global total_number_of_stable_patients
    global total_number_of_critical_patients
    global total_time_spent_in_queue


    # for deciding on stable or critical
    index = math.floor(random.random() * 10)  # generate uniform random number between 0-9 both inclusive.
    bag = ["S"] * 2 + ["C"] * 8  # 0.2 probability for S and 0.8 probability for C
    condition = bag[index]

    if condition == "S":  # stable
        total_number_of_stable_patients += 1
        healing_time = current_time + Generate_Home_Healing_Time("S")
        future_event_list.append(Event(f"DH{current_time}", future_event_list[0].id, healing_time))

    elif condition == "C":  # critical
        total_number_of_critical_patients += 1
        Found = False
        for i in range(K):
            if is_bed_available[i]:
                is_bed_available[i] = False
                bed_service_begin_time[i] = current_time
                next_departure_time = current_time + Generate_Hospital_Healing_Time()
                future_event_list.append(Event("DB" + str(i), future_event_list[0].id, next_departure_time))
                Found = True
                break
        if not Found:  # go home no available bed!
            total_number_of_rejected_due_to_bed += 1
            healing_time = current_time + Generate_Home_Healing_Time("C")
            future_event_list.append(Event(f"DH{current_time}", future_event_list[0].id, healing_time))

    # for departure from nurse
    nurse_id = int(future_event_list[0].type[2:])
    total_busy_time_for_nurse[nurse_id] += current_time - nurse_service_begin_time[nurse_id]
    is_nurse_available[nurse_id] = True
    total_number_of_departure_from_triage += 1

    # check whether queue is empty or not
    if queue_for_triage_nurse:
        next_departure_time = current_time + Generate_Nurse_Service_Time()
        is_nurse_available[nurse_id] = False
        nurse_service_begin_time[nurse_id] = current_time
        future_event_list.append(Event(future_event_list[0].type, queue_for_triage_nurse[0].id, next_departure_time))
        queue_for_triage_nurse.pop(0)

    future_event_list.pop(0)


# Bed-to-Departure
def Treated_at_Hospital():
    global future_event_list
    global total_time_healing_sick
    global is_bed_available
    global bed_service_begin_time
    global total_busy_time_for_bed
    global total_number_of_people_treated_in_hospital

    # for departure from bed
    bed_id = int(future_event_list[0].type[2:])
    total_time_healing_sick += current_time - bed_service_begin_time[bed_id]
    total_busy_time_for_bed[bed_id] += (current_time - bed_service_begin_time[bed_id])
    is_bed_available[bed_id] = True
    total_number_of_people_treated_in_hospital += 1

    future_event_list.pop(0)

def Treated_at_Home():
    global current_time
    global future_event_list
    global total_number_of_people_treated_in_home
    global total_time_healing_sick

    total_time_healing_sick += current_time - float(future_event_list[0].type[2:])
    total_number_of_people_treated_in_home += 1

    future_event_list.pop(0)


def Advance_Time():
    global future_event_list
    global current_time
    current_time = future_event_list[0].time


def Execute_Event(csv_file_name):
    global future_event_list
    global total_number_of_executed_events
    global total_time_spent_in_queue
    global total_time_there_is_atleast_one_empty_bed
    global total_time_there_is_atleast_one_empty_nurse
    global total_time_there_is_atleast_one_empty_nurse_and_bed

    future_event_list.sort(key=lambda Event: Event.time)
    if total_number_of_executed_events<=50:
        printCurrentEvent(csv_file_name)

    previous_time = current_time
    is_there_nurse_available = False
    is_there_bed_available = False
    for nurse in is_nurse_available:
        if nurse:
            is_there_nurse_available = True
            break
    for bed in is_bed_available:
        if bed:
            is_there_bed_available = True
            break
    length_of_queue_before_event = len(queue_for_triage_nurse)

    Advance_Time()
    if future_event_list[0].type == "A":  # Arrival
        Arrival()
    elif future_event_list[0].type.find("DN") != -1:  # Departure from Nurse
        Departure_Triage()
    elif future_event_list[0].type.find("DB") != -1:  # Departure from Bed
        Treated_at_Hospital()
    else: # Departure from Home
        Treated_at_Home()

    total_time_spent_in_queue += length_of_queue_before_event * (current_time - previous_time)

    if(is_there_nurse_available):
        total_time_there_is_atleast_one_empty_nurse += (current_time - previous_time)
    if(is_there_bed_available):
        total_time_there_is_atleast_one_empty_bed += (current_time - previous_time)
    if(is_there_nurse_available and is_there_bed_available):
        total_time_there_is_atleast_one_empty_nurse_and_bed += (current_time - previous_time)

    total_number_of_executed_events += 1

**calculate_model_response()** function is for **Part 2.2** in the description.

In [ ]:
def calculate_model_response(csv_file_name):
    global total_busy_time_for_nurse
    global total_busy_time_for_bed
    global current_time
    global total_number_of_arrival
    global total_number_of_rejected_due_to_bed
    global total_number_of_people_treated_in_home
    global total_time_healing_sick
    global total_number_of_people_treated_in_hospital
    global total_number_of_departure_from_triage
    global total_time_there_is_atleast_one_empty_nurse
    global total_time_there_is_atleast_one_empty_bed
    global total_time_there_is_atleast_one_empty_nurse_and_bed


    # Calculate marginal probability of being empty for triage and bed. Then, their joint probability.
    output1_triage_empty_probability = total_time_there_is_atleast_one_empty_nurse / current_time
    output1_bed_empty_probability = total_time_there_is_atleast_one_empty_bed / current_time
    output1_joint = total_time_there_is_atleast_one_empty_nurse_and_bed / current_time

    # average number of people that are rejected due to bed unavailability.
    output2 = total_number_of_rejected_due_to_bed / total_number_of_critical_patients

    # average utilization of each triage nurse
    output3 = [busy_time / current_time for busy_time in total_busy_time_for_nurse]

    # average number of occupied beds in the hospital
    output4 = sum(total_busy_time_for_bed) / current_time

    # average number of patients that are treated at home
    output5 = total_number_of_people_treated_in_home / (total_number_of_departure_from_triage)

    # average time a sick person gets better
    output6 = total_time_healing_sick / (
                total_number_of_people_treated_in_hospital + total_number_of_people_treated_in_home)

    csv_fields=["Output","Value"]

    csv_rows=[]
    csv_rows.append(["Long run marginal probabilities of being empty for triage",output1_triage_empty_probability])
    csv_rows.append(["Long run marginal probabilities of being empty for bed",output1_bed_empty_probability])
    csv_rows.append(["Long run marginal probabilities of being empty for triage and bed",output1_joint])

    csv_rows.append(["Average number of people that are rejected due to bed unavailability",output2])

    csv_rows.append(["Average utilization of each triage nurse",output3])

    csv_rows.append(["Average number of occupied beds in the hospital",output4])

    csv_rows.append(["Average number of patients that are treated at home",output5])

    csv_rows.append(["Average time a sick person gets better",output6])

    with open(csv_file_name,"w") as csv_file:
        csv.writer(csv_file).writerow(csv_fields)
        csv.writer(csv_file).writerows(csv_rows)

The following functions will be used to do simulation for **Empty, Half, Full** systems until **20, 200, 1000** patients are healed.

In [ ]:

def init_system(start_condition):
    global current_time
    global total_number_of_arrival
    global total_number_of_departure_from_triage
    global total_number_of_rejected_due_to_bed
    global total_number_of_people_treated_in_home
    global total_time_healing_sick
    global total_number_of_people_treated_in_hospital
    global queue_for_triage_nurse
    global is_nurse_available
    global total_busy_time_for_nurse
    global is_bed_available
    global total_busy_time_for_bed
    global future_event_list
    global nurse_service_begin_time
    global bed_service_begin_time
    global total_number_of_executed_events
    global total_number_of_people_in_queue
    global total_time_spent_in_queue
    global total_number_of_critical_patients
    global total_number_of_stable_patients
    global total_time_there_is_atleast_one_empty_nurse
    global total_time_there_is_atleast_one_empty_bed
    global total_time_there_is_atleast_one_empty_nurse_and_bed
    current_time = 0.0
    total_time_there_is_atleast_one_empty_nurse = 0.0
    total_time_there_is_atleast_one_empty_bed = 0.0
    total_time_there_is_atleast_one_empty_nurse_and_bed = 0.0

    # Arrival()
    total_number_of_arrival = 0  # Arrival to triage #total number of sick people
    total_number_of_people_in_queue = 0

    # Departure_Triage()
    total_number_of_departure_from_triage = 0  # Departure from triage to home or bed.
    total_number_of_rejected_due_to_bed = 0  # for average number of people that are rejected due to bed unavailability
    total_number_of_people_treated_in_home = 0  # for average number of patients that are treated at home
    total_time_healing_sick = 0.0  # for average time a sick person gets better.
    total_time_spent_in_queue = 0.0
    total_number_of_critical_patients = 0
    total_number_of_stable_patients = 0

    # Treated_at_Hospital()
    total_number_of_people_treated_in_hospital = 0

    # Execute_Event()
    total_number_of_executed_events = 0

    # Nurse
    queue_for_triage_nurse = []
    is_nurse_available = [True] * S
    nurse_service_begin_time = [0.0] * S
    total_busy_time_for_nurse = [0.0] * S  # for average utilization of each triage nurse

    # Bed
    is_bed_available = [True] * K
    bed_service_begin_time = [0.0] * K
    total_busy_time_for_bed = [0.0] * K

    # Future event List
    future_event_list = []

    future_event_list.append(Event("A", total_number_of_arrival + 1, Generate_Interarrival() + current_time))
    if start_condition == "Empty":
        pass
    elif start_condition == "Half":
        for i in range(S//2):
            is_nurse_available[i] = False
            nurse_service_begin_time[i] = current_time
            next_departure_time = current_time + Generate_Nurse_Service_Time()
            future_event_list.append(Event("DN" + str(i), i+1, next_departure_time))
        for i in range(K//2):
            is_bed_available[i] = False
            bed_service_begin_time[i] = current_time
            next_departure_time = current_time + Generate_Hospital_Healing_Time()
            future_event_list.append(Event("DB" + str(i),i+1, next_departure_time))

    elif start_condition == "Full":
        for i in range(S):
            is_nurse_available[i] = False
            nurse_service_begin_time[i] = current_time
            next_departure_time = current_time + Generate_Nurse_Service_Time()
            future_event_list.append(Event("DN" + str(i), i+1, next_departure_time))
        for i in range(K):
            is_bed_available[i] = False
            bed_service_begin_time[i] = current_time
            next_departure_time = current_time + Generate_Hospital_Healing_Time()
            future_event_list.append(Event("DB" + str(i), i+1, next_departure_time))


def calculate_last_changes():
    # Calculate nurse utility
    for i in range(S):
        if(not is_nurse_available[i]):
            total_busy_time_for_nurse[i] += current_time - nurse_service_begin_time[i]
    # Calculate bed utility
        if(not is_bed_available[i]):
            total_busy_time_for_bed[i] += current_time - bed_service_begin_time[i]

def init_simulation(start_condition,stop_condition):
    global total_number_of_people_treated_in_home
    global total_number_of_people_treated_in_hospital
    init_system(start_condition)

    #for writing events to .csv file
    csv_file_name=f"Event_{start_condition}_{stop_condition}.csv"
    with open(csv_file_name,"w") as csv_file:
        csv.writer(csv_file).writerow(csv_fields)

    while (total_number_of_people_treated_in_home + total_number_of_people_treated_in_hospital) < stop_condition:
        Execute_Event(csv_file_name)
    calculate_last_changes()

    #for writing model response to .csv file
    calculate_model_response(f"Model_{start_condition}_{stop_condition}.csv")
    print(f"{start_condition}-{stop_condition} simulation is done!")


def do_all_simulations():
    [init_simulation(start, stop) for start in ["Empty", "Half", "Full"] for stop in [20, 200, 1000]]


#init_simulation("Empty", 20)
do_all_simulations()